In [10]:
import cvxpy as cp
import numpy as np

# ---------- Problem size ----------
N = 9  # i in [9] = {1,...,9}

# ---------- Parameters (22nm CMOS) ----------
beta_n = 3.9e-3     # A/V^2
beta_p = 1.38e-3    # A/V^2
lam    = 0.05       # channel length modulation λ
Vth    = 0.5        # V_th
RL     = 10e3       # load resistor 10 kΩ
A_req  = 500.0      # required minimum gain

# Size bounds (all i share same bounds; 可以按需要改成向量)
W_min = 12e-9       # 12 nm
W_max = 10e-3       # 10 mm
L_min = 15e-9       # 15 nm
L_max = 10e-3       # 10 mm

# ---------- Decision variables (all positive for GP) ----------
I    = cp.Variable(N,  pos=True, name="I")        # I_i
Vov  = cp.Variable(N,  pos=True, name="Vov")      # V_{ov,i}
dV   = cp.Variable(N,  pos=True, name="dV")       # ΔV_i
s    = cp.Variable(N,  pos=True, name="s")        # s_i
W    = cp.Variable(N,  pos=True, name="W")        # W_i
L    = cp.Variable(N,  pos=True, name="L")        # L_i
Vdd  = cp.Variable(pos=True, name="Vdd")          # V_dd

constraints = []
ineqs = []   # (name, lhs, rhs, constraint) for all inequality constraints

# ---------- Basic constraints ----------

# W_i / (L_i s_i) = 1   (monomial equality)
for i in range(N):
    constraints.append(W[i] * (L[i] * s[i])**-1 == 1)

# Vdd / 12 <= 1  -> Vdd <= 12
lhs = Vdd
rhs = 12.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("Vdd_max", lhs, rhs, c))

# Width bounds:  W_min <= W_i <= W_max   (写成GP友好的形式)
for i in range(N):
    # W_min / W_i <= 1
    lhs = W_min * W[i]**-1
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    # ineqs.append((f"W_min_{i}", lhs, rhs, c))

    # W_i / W_max <= 1
    lhs = W[i] * (W_max**-1)
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    # ineqs.append((f"W_max_{i}", lhs, rhs, c))

# Length bounds: L_min <= L_i <= L_max
for i in range(N):
    # L_min / L_i <= 1
    lhs = L_min * L[i]**-1
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    # ineqs.append((f"L_min_{i}", lhs, rhs, c))

    # L_i / L_max <= 1
    lhs = L[i] * (L_max**-1)
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    # ineqs.append((f"L_max_{i}", lhs, rhs, c))

# ---------- Current constraints ----------

# Sets of indices (convert 1-based {1,...,9} -> 0-based python)
idx_n = [0, 1, 4, 5, 6]      # i in {1,2,5,6,7}  -> NMOS
idx_p = [2, 3, 7, 8]         # i in {3,4,8,9}   -> PMOS

# (1/2) β_{n/p} s_i Vov_i^2 (1 + λ(Vov[i] + ΔV[i])) I_i^{-1} <= 1,  i in [9]
for i in idx_n:
    lhs = 0.5 * beta_n * s[i] * Vov[i]**2 * (1 + lam*(Vov[i] + dV[i])) * I[i]**-1
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    ineqs.append((f"I{i}_def_n", lhs*I[i], I[i], c))

for i in idx_p:
    lhs = 0.5 * beta_p * s[i] * Vov[i]**2 * (1 + lam*(Vov[i] + dV[i])) * I[i]**-1
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    ineqs.append((f"I{i}_def_p", lhs*I[i], I[i], c))

# (I1 + I2) / I6 <= 1  -> (I[0] + I[1]) * I[5]^{-1} <= 1
lhs = (I[0] + I[1]) * I[5]**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("I12_over_I6", lhs, rhs, c))

# I_i / I_i' = 1 for specified pairs (convert to monomial equalities)
current_equal_pairs = [(8, 4), (0, 2), (1, 3), (6, 7)]
for i, j in current_equal_pairs:
    constraints.append(I[i] * I[j]**-1 == 1)

# ---------- Voltage equalities ----------

# ΔV_i = V_th, i in {3,5,9} -> 0-based {2,4,8}
for i in [2, 4, 8]:
    constraints.append(dV[i] == Vth)

# Vov,5 = Vov,6 = Vov,7 -> indices 4,5,6
constraints.append(Vov[4] == Vov[5])
constraints.append(Vov[5] == Vov[6])

# Vov,1 = Vov,2 -> indices 0,1
constraints.append(Vov[0] == Vov[1])

# Vov,3 = Vov,4 -> indices 2,3
constraints.append(Vov[2] == Vov[3])

# ---------- Vdd-related inequalities ----------

# 2(Vov2 + Vov4 + dV2 + dV4) / Vdd <= 1
lhs = 2*(Vov[1] + Vov[3] + dV[1] + dV[3]) * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("swing_1", lhs, rhs, c))

# (Vov7 + Vov8 + dV7 + dV8) / Vdd <= 1
lhs = (Vov[6] + Vov[7] + dV[6] + dV[7]) * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("swing_2", lhs, rhs, c))

# 2(Vov1 + Vov6 + dV6 + Vth) / Vdd <= 1
lhs = 2*(Vov[0] + Vov[5] + dV[5] + Vth) * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("swing_3", lhs, rhs, c))

# 2(dV2 + Vov8) / Vdd <= 1
lhs = 2*(dV[1] + Vov[7]) * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("swing_4", lhs, rhs, c))

# 2(dV1 + Vov3) / Vdd <= 1
lhs = 2*(dV[0] + Vov[2]) * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("swing_5", lhs, rhs, c))

# ---------- Gain constraint ----------
# A_req (λ I2 + λ I4)(λ I7 + λ I8 + 1/RL) / (β_n β_p s1 s8 Vov1 Vov8) <= 1
num = A_req * (lam*I[1] + lam*I[3]) * (lam*I[6] + lam*I[7] + 1.0/RL)
den = beta_n * beta_p * s[0] * s[7] * Vov[0] * Vov[7]
lhs = num * den**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("gain", (den*A_req)/num, A_req, c))

# ---------- 辅助约束（改成 GP 合法形式） ----------
# I[i] >= 0.0001  ->  0.01 * I[i]^{-1} <= 1
LOWER_BOUND = 0.0001
for i in range(N):
    lhs = LOWER_BOUND * I[i]**-1
    rhs = 1.0
    c = lhs <= rhs
    constraints.append(c)
    ineqs.append((f"I_min_{i}", I[i], LOWER_BOUND, c))

# Vdd >= 0.01 -> 0.01 * Vdd^{-1} <= 1
lhs = LOWER_BOUND * Vdd**-1
rhs = 1.0
c = lhs <= rhs
constraints.append(c)
ineqs.append(("Vdd_min", Vdd, LOWER_BOUND, c))

# ---------- Objective ----------
# min sum_{i in {5,6,8}} I_i * Vdd  -> indices {4,5,7}  (1-based 5,6,8)
obj = cp.Minimize((I[4] + I[5] + I[7]) * Vdd)

problem = cp.Problem(obj, constraints)

# ---------- Solve as a Geometric Program ----------
problem.solve(gp=True)   # 可以根据环境加 solver=cp.MOSEK 等

print("status  :", problem.status)
print("optimal value :", problem.value)
print("Vdd*       :", Vdd.value)
print("I*         :", I.value)
print("Vov*       :", Vov.value)
print("dV*        :", dV.value)
print("s*         :", s.value)
print("W*         :", W.value)
print("L*         :", L.value)

# ---------- 打印每个不等式约束的左右两边数值 ----------
if problem.status in ["optimal", "optimal_inaccurate"]:
    print("\n===== Inequality constraints (lhs, rhs, active?) =====\n")
    for name, lhs, rhs, c in ineqs:
        lhs_val = lhs.value
        rhs_val = rhs if np.isscalar(rhs) else rhs.value
        # 有些可能是数组（但这里我们都是标量），保险起见处理一下
        lhs_val_float = float(np.max(lhs_val))
        rhs_val_float = float(np.max(rhs_val))
        active = np.isclose(lhs_val_float, rhs_val_float, atol=1e-4)
        print(f"{name:15s}:  lhs = {lhs_val_float:.4e},  rhs = {rhs_val_float:.4e},  active = {active}")
else:
    print("\nNo optimal solution; cannot evaluate inequality sides.")


status  : optimal
optimal value : 0.0004000000097601724
Vdd*       : 1.000000016396642
I*         : [0.0001 0.0001 0.0001 0.0001 0.0001 0.0002 0.0001 0.0001 0.0001]
Vov*       : [3.35944215e-09 3.35944215e-09 3.16069281e-04 3.16069281e-04
 1.24858875e-09 1.24858875e-09 1.24858875e-09 2.29940979e-04
 1.08361454e-02]
dV*        : [5.64444534e-05 3.77284059e-05 5.00000000e-01 1.11765297e-04
 5.00000000e-01 6.82207766e-11 1.97128297e-05 4.53919279e-05
 5.00000000e-01]
s*         : [5.82010562e+05 1.22825069e+03 4.68701333e+00 1.10857723e+01
 8.25333076e+02 1.10792817e+03 1.00554458e+03 5.58818916e+05
 4.14200839e-02]
W*         : [9.30469427e-03 8.74995317e-05 4.36604539e-06 6.83919297e-06
 7.01390568e-05 8.28638616e-05 7.83949494e-05 9.03984997e-03
 4.02228716e-07]
L*         : [1.59871571e-08 7.12391469e-08 9.31519729e-07 6.16934284e-07
 8.49827286e-08 7.47917271e-08 7.79626793e-08 1.61767072e-08
 9.71095851e-06]

===== Inequality constraints (lhs, rhs, active?) =====

Vdd_max        :  